In [1]:
#!pip install -q wot

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import ipywidgets as widgets
import wot

Load transport map model and cell sets

In [3]:
tmap_model = wot.tmap.TransportMapModel.from_directory('tmaps/serum')
cell_sets = wot.io.read_sets('data/cell_sets.gmt', as_dict=True)

Convert cell sets indicating whether a cell at time t is in a cell set

In [4]:
populations = tmap_model.population_from_cell_sets(cell_sets, at_time=18)

Print cell set size

In [5]:
for pop in populations:
    print('{}, {}'.format(pop.name, np.count_nonzero(pop.p)))

IPS, 412
Stromal, 1239
Neural, 819
Epithelial, 503
Epithelial_clust, 1013
Trophoblast, 292
OPC, 141
Astrocyte, 112
Neuron, 122
RG_Gdf10, 144
RG_Id3, 160
RG_Neurog2, 21
RG, 255
OtherNeural, 390
TrophoLarger, 1305


Compute trajectories. Trajectory dataset has cells on rows and cell set populations on columns

In [6]:
trajectory_ds = tmap_model.compute_trajectories(populations)

Load embedding coordinates

In [7]:
coord_df = pd.read_csv('data/fle_coords.txt', sep='\t', index_col=0)
trajectory_ds.obs = trajectory_ds.obs.join(coord_df)

In [8]:
trajectory_ds.X[0]

array([0.00017564, 0.00020802, 0.00018117, 0.00018206, 0.00018167,
       0.00017837, 0.0001835 , 0.0001826 , 0.00018003, 0.00018134,
       0.00018281, 0.00018048, 0.00018189, 0.00018085, 0.00018093],
      dtype=float32)

Visualize trajectories

In [9]:
trajectory_dropdown = widgets.Dropdown(
    options=trajectory_ds.var.index,
    description='Trajectories:'
)

def update_trajectory_vis(name):
    figure = plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.tight_layout()
    plt.title(name)
    plt.scatter(coord_df['x'], coord_df['y'], c='#f0f0f0',
                   s=4, marker=',', edgecolors='none', alpha=0.8)
    plt.scatter(trajectory_ds.obs['x'], trajectory_ds.obs['y'], c=trajectory_ds[:, name].X,
                   s=4, marker=',', edgecolors='none')
    plt.colorbar().ax.set_title('Trajectory')

widgets.interact(update_trajectory_vis, name=trajectory_dropdown)

interactive(children=(Dropdown(description='Trajectories:', options=('IPS', 'Stromal', 'Neural', 'Epithelial',…

<function __main__.update_trajectory_vis(name)>

Compute cell fates

In [10]:
fate_ds = tmap_model.compute_fates(populations)

Visualize cell fates

In [13]:
fate_dropdown1 = widgets.Dropdown(
    options=fate_ds.var.index,
    description='Fate 1:'
)
fate_dropdown2 = widgets.Dropdown(
    options=fate_ds.var.index,
    description='Fate 2:',
    value=fate_ds.var.index[1]
)

def update_fate_vis(name1, name2):
    figure = plt.figure(figsize=(10, 10))
    fate1 = fate_ds[:, name1].X
    fate2 = fate_ds[:, name2].X
    p = np.log(1np.divide(fate1, fate2, out=np.zeros_like(fate1), where=fate2 != 0))
    plt.scatter(fate_ds.obs['day'], p, s=4, marker=',')
    plt.xlabel('Day')
    plt.ylabel('Log Odds')
    plt.title('{} vs. {}'.format(name1, name2))

widgets.interact(update_fate_vis, name1=fate_dropdown1, name2=fate_dropdown2)

interactive(children=(Dropdown(description='Fate 1:', options=('IPS', 'Stromal', 'Neural', 'Epithelial', 'Epit…

<function __main__.update_fate_vis(name1, name2)>